<a href="https://colab.research.google.com/github/sciencenerd880/HoopsTalk/blob/main/openai4o_commentary_JS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook consists of

(i) using openAI_4o API to perform commentary generation

In [1]:
#@title Codes to mount your google drive folder
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab_Notebooks/SMU_MITB_NLP/group_project/notebooks/

Mounted at /content/drive
/content/drive/MyDrive/Colab_Notebooks/SMU_MITB_NLP/group_project/notebooks


In [2]:
#@title Pip install requirements.txt
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [3]:
!pip install openai

In [5]:
#@title Add Cuda Device
import torch
device = 0 if torch.cuda.is_available() else -1  # device 0 for the first GPU
device

0

In [6]:
#@title Import necessary libraries
import os
import logging
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd

import openai
import logging

import cv2
import base64

from tqdm import tqdm
#from video_processor import extract_encode_frames, create_message
#from openai_client import get_commentary_for_frames

In [ ]:
cd ..

/content/drive/MyDrive/Colab_Notebooks/SMU_MITB_NLP/group_project/notebooks


In [7]:
import glob

for path in glob.glob('./data/raw/NSVA_Data/NSVA_Video/*.mp4'):
    print(path)

./data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10977.mp4
./data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10980.mp4
./data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10978.mp4
./data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10979.mp4
./data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10982.mp4
./data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10981.mp4
./data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10984.mp4
./data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10983.mp4
./data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10986.mp4
./data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10985.mp4
./data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10988.mp4
./data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10987.mp4


In [9]:
video_dir = './data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx'
abs_video_dir = Path(video_dir).resolve()
for path in abs_video_dir.glob('*.mp4'):
    print(path)

In [22]:
df = pd.read_csv('./data/raw/NSVA_Data/NSVA_Data/0021800013-dal-vs-phx.csv')
print(df.columns)
#iterate df row
for index, row in df.iterrows():
    #containing duplicate
    if "</>" in (row['video_url']):
        #split by </>
        split_url = row['video_url'].split(' </> ')
        #take only the first
        print(index,split_url[0],row['playerName'])

    #assumes to have only one url
    else:
      print(index,row['video_url'],row['playerName'])

Index(['actionNumber', 'actionType', 'clock', 'description', 'isFieldGoal',
       'location', 'period', 'personId', 'playerName', 'playerNameI',
       'pointsTotal', 'scoreAway', 'scoreHome', 'shotDistance', 'shotResult',
       'subType', 'teamId', 'teamTricode', 'videoAvailable', 'xLegacy',
       'yLegacy', 'video_url', 'video_id', 'data_split_id',
       'delexicalized_description', 'players_in_description',
       'action_type_code'],
      dtype='object')
0 https://videos.nba.com/nba/pbp/media/2018/10/17/0021800013/4/4370f4e5-5808-8024-f98f-c5f67855904f_1280x720.mp4 Ayton
1 https://videos.nba.com/nba/pbp/media/2018/10/17/0021800013/7/4d9c01e8-f1a3-0385-fafb-1f2a3543b81e_1280x720.mp4 Matthews </> Ayton
2 https://videos.nba.com/nba/pbp/media/2018/10/17/0021800013/9/afa37cc3-86c1-5776-0d79-6c072beda54e_1280x720.mp4 Booker </> Doncic
3 https://videos.nba.com/nba/pbp/media/2018/10/17/0021800013/11/f8f23ea6-c986-8ca6-4cb7-e23567610783_1280x720.mp4 Jordan
4 https://videos.nba.com/nba/

In [ ]:
#@title Step 1) Useful Functions

def main():
    #load_dotenv()

    frame_interval = 60
    video_dir = './data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx'
    text_dir = './data/text/GPT4o/commentary_results.csv'
    #text_dir = 'commentary_results.csv'
    #api_key = os.getenv('OPENAI_API_KEY')

    from google.colab import userdata
    api_key = userdata.get('hoopstalk')
    #print('check ur api key',userdata.get('hoopstalk'))

    if not api_key:
        logging.error("API key not found. Make sure to set the OPENAI_API_KEY environment variable.")
        return

    # Convert the directory string to a Path object
    abs_video_dir = Path(video_dir).resolve()
    logging.info(f"Absolute video directory: {abs_video_dir}")

    results = []
    commentary_buffer = []  # Initialize a buffer to store the last 10 lines of commentary

    # Use glob to find all .mp4 files in the directory

    video_files = glob.glob('./data/raw/NSVA_Data/NSVA_Video/*.mp4')

    for video_file in tqdm(video_files, desc="Processing Videos"):
    #for video_file in glob.glob('./data/raw/NSVA_Data/NSVA_Video/*.mp4'):
        print('mp4',video_file)
        logging.info(f"Processing video: {video_file}")
        base64_frames = extract_encode_frames(video_file, frame_interval)
        messages = create_message(base64_frames, frame_interval, "\n".join(commentary_buffer))
        commentary = get_commentary_for_frames(api_key, messages)

        if commentary:
            #results.append({'video_file': video_file.name, 'commentary': commentary})
            results.append({'video_file': video_file, 'commentary': commentary})
            commentary_buffer.extend(commentary.split("\n"))  # Update the buffer with new lines
            commentary_buffer = commentary_buffer[-10:]  # Keep only the last 10 lines
        else:
            logging.error(f"Failed to get commentary for {video_file.name}")

    # Save results to CSV
    print('results',results)
    df = pd.DataFrame(results)
    print(text_dir)
    df.to_csv(text_dir, index=False)
    logging.info(f"Commentary saved to {text_dir}")

def get_commentary_for_frames(api_key, messages, max_tokens=300, model="gpt-4o"):
    openai.api_key = api_key

    try:
        response = openai.chat.completions.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens
        )
        # logging.info(f"GPT Response: {response.choices[0].message.content}")
        return response.choices[0].message.content
    except openai.error.OpenAIError as e:
        logging.error(f"OpenAI API error: {e}")
        return None

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def extract_encode_frames(video_path, frame_interval=50):
    video_path = str(video_path).replace('\\', '/')
    video = cv2.VideoCapture(video_path)

    base64_frames = []
    frame_count = 0
    while video.isOpened():
        success, frame = video.read()
        if not success:
            break
        if frame_count % frame_interval == 0:
            _, buffer = cv2.imencode(".jpg", frame)
            base64_frames.append(base64.b64encode(buffer).decode("utf-8"))
        frame_count += 1

    video.release()

    return base64_frames

def create_message(encoded_frames, frame_interval=50, previous_commentary=""):
    content = [
        "You are an NBA commentator. Here are frames from a recent NBA game. Your task is to provide a brief and engaging commentary summarizing the key events depicted in these frames. Please maintain a flow and continuity with the previous commentary provided. Ensure that the events described are logically sequenced and contextually appropriate for different parts of the game:",
        *map(lambda x: {"image": x, "resize": 768}, encoded_frames[0::frame_interval]),
        f"Previous commentary:\n{previous_commentary}",
        "Consider the following types of events to help structure your commentary:",
        """
        1. Play Start: Describe the beginning of an offensive or defensive play.
        2. Shot Attempt: [Player] attempts a [shot type] from [distance].
        3. Rebound: [Player] grabs the rebound.
        4. Successful Shot: [Player] scores a [shot type] from [distance].
        5. Assist: Assist by [Player].
        6. Foul: [Player] commits a [foul type].
        7. Turnover: [Player] loses the ball.
        8. Significant Moment: Describe any significant moment like a game-changing play or a remarkable defensive action.
        9. End of Period: End of [period number].
        Combine these elements to provide a cohesive and engaging summary of the key events depicted in these frames in approximately two sentences.
        """
    ]

    messages = [{"role": "user", "content": content}]

    return messages


In [ ]:
logging.basicConfig(level=logging.INFO)
main()

Processing Videos:   0%|          | 0/12 [00:00<?, ?it/s]

mp4 ./data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10977.mp4


Processing Videos:   8%|▊         | 1/12 [00:07<01:19,  7.25s/it]

mp4 ./data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10980.mp4


Processing Videos:  17%|█▋        | 2/12 [00:16<01:21,  8.19s/it]

...

mp4 ./data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10988.mp4


Processing Videos:  92%|█████████▏| 11/12 [01:26<00:07,  7.67s/it]

mp4 ./data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10987.mp4


Processing Videos: 100%|██████████| 12/12 [01:33<00:00,  7.76s/it]

results [{'video_file': './data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10977.mp4', 'commentary': '"And we\'re off! The crowd roars as the tip-off gets underway at the Talking Stick Resort Arena, setting the stage for what promises to be an exciting game."'}, {'video_file': './data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10980.mp4', 'commentary': '"And here we go with the Suns setting up their first offensive play. Booker dishes it out to Ayton in the paint, Ayton goes up for a contested layup—it\'s good! Phoenix strikes first, putting two on the board, and the crowd is already on their feet!"'}, {'video_file': './data/raw/NSVA_Data/NSVA_Video/0021800013-dal-vs-phx_10978.mp4', 'commentary': '"And now it\'s the Mavericks\' turn. Doncic handles the ball at the top of the key, looking for an opening—he finds Barnes cutting to the basket! Barnes attempts a layup, and it\'s a smooth finish. Dallas quickly answers back, tying it up 2-2 with just under a minute into the game—it\